In [ ]:
import numpy as np
import scipy.io
from itertools import combinations
from scipy.special import comb

In [ ]:
def import_data(patient, N=3):
    '''...'''
    data = scipy.io.loadmat(f'/Users/anibal/Google Drive/Information cochains/O-info/OinfoSinfo/OinfoCopulasN{N}/Oinfop{patient}.mat')

    del data['__header__']
    del data['__version__']
    del data['__globals__']

    data['TC']  = (data['Sinfo'] + data['Oinfo'])/2
    data['DTC'] = (data['Sinfo'] - data['Oinfo'])/2
    
    return data

In [ ]:
def construct_boundary(d, r=19):
    '''...'''
    domain_basis = tuple(combinations(range(r+1), d+1))
    target_basis = tuple(combinations(range(r+1), d))
    target_basis_ix = {tuple(v): index for index, v in enumerate(target_basis)}
    N = comb(r+1, d+1, exact=True)
    M = comb(r+1, d, exact=True)
    D = scipy.sparse.csr_matrix((M, N), dtype=np.int8)
    for j in range(d+1):
        jth_faces_ix = [
                target_basis_ix[tuple(np.concatenate((l[:j], l[j+1:])))]
                for l in domain_basis
                ]
        D += scipy.sparse.csr_matrix ( 
                        ([(-1)**j]*N, (jth_faces_ix, range(N))), 
                        shape=(M, N), dtype=np.int8
                        )
    return D

def construct_coboundary(d, r=19):
    '''...'''
    return construct_boundary(d+1,r).T

In [ ]:
import scipy.sparse.linalg

def construct_laplacian(d, r=19, weights=None):
    '''...'''
    B_from = construct_boundary(d,r)
    BT_to = construct_coboundary(d-1,r)
    L_down = np.multiply(BT_to, B_from).astype(np.float)
    B_to = construct_boundary(d+1,r)
    BT_from = construct_coboundary(d,r)
    L_up = np.multiply(B_to, BT_from).astype(np.float)
    return L_down + L_up

laplacian = construct_laplacian(2,3)
print(laplacian.shape)
print(comb(3+1, 2+1, exact=True))
eigens = scipy.sparse.linalg.eigs(laplacian, k=laplacian.shape[1]-2)